#📌 Extracão

In [111]:
#importando as biblioteca que serão utilizadas na analise
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [112]:
url = 'https://raw.githubusercontent.com/alura-cursos/challenge2-data-science/refs/heads/main/TelecomX_Data.json'
df = pd.read_json(url)

In [113]:
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


#🔧 Transformação

In [114]:
# Normalização das colunas customer, phone, internet e account
df_norm = pd.concat([
    df.drop(columns=['customer', 'phone', 'internet', 'account']),
    pd.json_normalize(df['customer']),
    pd.json_normalize(df['phone']),
    pd.json_normalize(df['internet']),
    pd.json_normalize(df['account'])
], axis=1)
df_norm.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [123]:
# verificando existencias de valores nulos

df_norm.isnull().sum()
print(f"\nTotal de valores ausentes: {df_norm.isnull().sum().sum()}")


Total de valores ausentes: 0


In [116]:
# verificando existencia de valores duplicados

duplicados = df_norm.duplicated().sum()
print(f"Total de linhas duplicadas: {duplicados}")
if duplicados > 0:
    print("Linhas duplicadas:")
    print(df_norm[df_norm.duplicated()])

Total de linhas duplicadas: 0


In [117]:
# verificando tipos de dados por colona
print("Tipos de dados por coluna:")
df_norm.dtypes

Tipos de dados por coluna:


customerID           object
Churn                object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
Charges.Monthly     float64
Charges.Total        object
dtype: object

In [118]:
# verificicando a existencia de Possíveis inconsistências nas colunas

for col in colunas_object:
    valores_unicos = df_norm[col].value_counts()
    print(f"\nColuna '{col}':")
    print(f"Valores únicos: {len(valores_unicos)}")
    print("Distribuição dos valores:")
    print(valores_unicos)

    # Verificar possíveis inconsistências (valores similares)
    valores_str = df_norm[col].dropna().astype(str).str.lower().str.strip()
    if len(valores_str.unique()) != len(df_norm[col].dropna().unique()):
        print(f"⚠️  Possível inconsistência detectada em '{col}' (diferenças em maiúsculas/minúsculas ou espaços)")


Coluna 'customerID':
Valores únicos: 7267
Distribuição dos valores:
customerID
9995-HOTOH    1
0002-ORFBO    1
0003-MKNFE    1
9970-QBCDA    1
9968-FFVVH    1
             ..
0014-BMAQU    1
0013-SMEOE    1
0013-MHZWF    1
0013-EXCHZ    1
0011-IGKFF    1
Name: count, Length: 7267, dtype: int64

Coluna 'Churn':
Valores únicos: 3
Distribuição dos valores:
Churn
No     5174
Yes    1869
        224
Name: count, dtype: int64

Coluna 'gender':
Valores únicos: 2
Distribuição dos valores:
gender
Male      3675
Female    3592
Name: count, dtype: int64

Coluna 'Partner':
Valores únicos: 2
Distribuição dos valores:
Partner
No     3749
Yes    3518
Name: count, dtype: int64

Coluna 'Dependents':
Valores únicos: 2
Distribuição dos valores:
Dependents
No     5086
Yes    2181
Name: count, dtype: int64

Coluna 'PhoneService':
Valores únicos: 2
Distribuição dos valores:
PhoneService
Yes    6560
No      707
Name: count, dtype: int64

Coluna 'MultipleLines':
Valores únicos: 3
Distribuição dos valores:
Mu

In [119]:
# verificando quais colunas tem valor numerico

colunas_numericas = df_norm.select_dtypes(include=['number']).columns
if len(colunas_numericas) > 0:
    print(df_norm[colunas_numericas].describe())
else:
    print("Nenhuma coluna numérica encontrada")

       SeniorCitizen       tenure  Charges.Monthly
count    7267.000000  7267.000000      7267.000000
mean        0.162653    32.346498        64.720098
std         0.369074    24.571773        30.129572
min         0.000000     0.000000        18.250000
25%         0.000000     9.000000        35.425000
50%         0.000000    29.000000        70.300000
75%         0.000000    55.000000        89.875000
max         1.000000    72.000000       118.750000


In [120]:
# verificando quais colunas numericas tem valor zero ou negativo

for col in colunas_numericas:
    valores_negativos = (df_norm[col] < 0).sum()
    if valores_negativos > 0:
        print(f"⚠️  Coluna '{col}' tem {valores_negativos} valores negativos")

    valores_zero = (df_norm[col] == 0).sum()
    if valores_zero > 0:
        print(f"ℹ️  Coluna '{col}' tem {valores_zero} valores zero")

ℹ️  Coluna 'SeniorCitizen' tem 6085 valores zero
ℹ️  Coluna 'tenure' tem 11 valores zero


In [121]:
# Conversão de colunas com valores 0 e 1 para "No" e "Yes" 
for col in df_norm.columns:
    # Verificar se a coluna contém apenas valores 0 e 1 (ignorando NaN)
    valores_unicos = df_norm[col].dropna().unique()
    if len(valores_unicos) <= 2 and all(val in [0, 1] for val in valores_unicos):
        print(f"Convertendo coluna '{col}': 0 → No, 1 → Yes")
        df_norm[col] = df_norm[col].replace({0: "No", 1: "Yes"})
        
print("Conversão concluída!")
df_norm.head(10) # verificando a alteração

Convertendo coluna 'SeniorCitizen': 0 → No, 1 → Yes
Conversão concluída!


,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,No,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.3
1,0003-MKNFE,No,Male,No,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.4
2,0004-TLHLJ,Yes,Male,No,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,Yes,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,Yes,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.4
5,0013-MHZWF,No,Female,No,No,Yes,9,Yes,No,DSL,...,No,No,Yes,Yes,Yes,Month-to-month,Yes,Credit card (automatic),69.40,571.45
6,0013-SMEOE,No,Female,Yes,Yes,No,71,Yes,No,Fiber optic,...,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),109.70,7904.25
7,0014-BMAQU,No,Male,No,Yes,No,63,Yes,Yes,Fiber optic,...,No,No,Yes,No,No,Two year,Yes,Credit card (automatic),84.65,5377.8
8,0015-UOCOJ,No,Female,Yes,No,No,7,Yes,No,DSL,...,No,No,No,No,No,Month-to-month,Yes,Electronic check,48.20,340.35
9,0016-QLJIS,No,Female,No,Yes,Yes,65,Yes,Yes,DSL,...,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Mailed check,90.45,5957.9


,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,No,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.3
1,0003-MKNFE,No,Male,No,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.4
2,0004-TLHLJ,Yes,Male,No,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,Yes,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,Yes,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.4
5,0013-MHZWF,No,Female,No,No,Yes,9,Yes,No,DSL,...,No,No,Yes,Yes,Yes,Month-to-month,Yes,Credit card (automatic),69.40,571.45
6,0013-SMEOE,No,Female,Yes,Yes,No,71,Yes,No,Fiber optic,...,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),109.70,7904.25
7,0014-BMAQU,No,Male,No,Yes,No,63,Yes,Yes,Fiber optic,...,No,No,Yes,No,No,Two year,Yes,Credit card (automatic),84.65,5377.8
8,0015-UOCOJ,No,Female,Yes,No,No,7,Yes,No,DSL,...,No,No,No,No,No,Month-to-month,Yes,Electronic check,48.20,340.35
9,0016-QLJIS,No,Female,No,Yes,Yes,65,Yes,Yes,DSL,...,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Mailed check,90.45,5957.9


#📊 Carga e análise

#📄Relatorio Final